In [1]:
# 对应slim文件夹
# 运行得到路径下的各种.tfrecord文件                      把原始图片转化成.tfrecord文件      image_to_tfexample
# G:\Anaconda3\My_Jupyter\slim\datasets\myimages.py      是读取.tfrecord文件取内存中        keys_to_features
# 修改train_image_classifier
# 写一个批处理文件train.bat
# https://blog.csdn.net/gubenpeiyuan/article/details/80284888

In [ ]:
# 用python执行我们改好的这个程序train_image_classifier.py
# python G:/Anaconda3/My_Jupyter/slim/train_image_classifier.py ^
# 模型存放在
# --train_dir=G:\Anaconda3\My_Jupyter\slim\model_save_at ^
# 
# --dataset_name=myimages ^
# --dataset_split_name=train ^
# 图片存放的位置
# --dataset_dir=G:\Anaconda3\My_Jupyter\slim\images ^
# --batch_size=10 ^
# --max_number_of_steps=10000 ^
# --model_name=inception_v3 ^
# pause

In [2]:
import tensorflow as tf
import os
import random
import sys
import math

In [3]:
# 验证集数量
_NUM_TEST = 500
# 随机种子
_RANDOM_SEED = 0
# 数据块 把图片进行分割，对于数据量比较大的时候使用  
_NUM_SHARDS = 5
# 数据块路径 其实就是存放图片的路径
DATASET_DIR = 'G:/Anaconda3/My_Jupyter/slim/images'
# 标签文件名称
LABELS_FILENAME = 'G:/Anaconda3/My_Jupyter/slim/images/labels.txt'


# 预处理是为了生成一些 .tfrecord 的文件 它是tf官方提供的底层是 protobuf 的文件 （一种谷歌开源的 二进制的 文件存储方式 数据传输效率高）

# 定义.tfrecord文件的路径+名字
# 把我们的图片一张张变成.tfrecord文件 调用的时候也不是说直接调用图片 而是调用这个文件
def _get_dataset_filename(dataset_dir,split_name,shard_id):
    output_filename = 'image_%s_%05d-of-%05d.tfrecord' % (split_name,shard_id,_NUM_SHARDS)
    return os.path.join(dataset_dir,output_filename)

# 判断 tfrecord 文件是否存在
def _dataset_exists(dataset_dir):
    for split_name in ['train','test']:
        for shard_id in range(_NUM_SHARDS):
            # 定义.tfrecord文件的路径+名字
            output_filename = _get_dataset_filename(dataset_dir,split_name,shard_id)
        if not tf.gfile.Exists(output_filename):
            return False
    return True
    
    
    
    
# 获取所有文件以及分类
# 传入一个路径dataset_dir 在这个路径下找所有文件夹filename 合并路径path = dataset_dir filename 
# 如果这个路径是目录 则保存此路径在directories = []中    并保存这个文件夹名即图片集分类名在class_names = []中
# 循环于每一个文件夹 

# 最终返回图片的绝对地址列表  photo_filenames = [] 和类名列表    class_names = []
def _get_filenames_and_classes(dataset_dir):  
    # dataset_dir = DATASET_DIR = 'G:/Anaconda3/My_Jupyter/retrain/data/train' 
    # 数据目录 如               
    directories = []  # path   G:/Anaconda3/My_Jupyter/retrain/data/train/airplane
    # 分类名称 如               
    class_names = []  # filename                                          airplane
    
    for filename in os.listdir(dataset_dir):
        # 合并文件路径
        path = os.path.join(dataset_dir,filename)
        # 判断该路径是否为目录
        if os.path.isdir(path):
            # 加入数据目录
            directories.append(path)
            # 加入类别名称
            class_names.append(filename)
            
    photo_filenames = []
    # 循环每个分类文件夹  如在G:/Anaconda3/My_Jupyter/retrain/data/train/airplane文件夹中 循环得到图片名称 filename
    # 合并path = G:/Anaconda3/My_Jupyter/retrain/data/train/airplane/0001.jpg
    # 把这个图片的绝对地址存在   photo_filenames = []  中
    for directory in directories:
        for filename in os.listdir(directory):
            path = os.path.join(directory,filename)
            # 把图片加入图片列表
            photo_filenames.append(path)
    return photo_filenames, class_names
        
    
    
# 文本转化为整数格式
def int64_feature(values):
    if not isinstance(values,(tuple,list)):
        values = [values]
         # print(values)  
    return tf.train.Feature(int64_list = tf.train.Int64List(value = values))
    
# 文本转化为字节格式
def bytes_featurn(values):
    return tf.train.Feature(bytes_list = tf.train.BytesList(value = [values]))


# 转化tfexample 固定写法
def image_to_tfexample(image_data,image_format,class_id):
    # abstract base class for protocol messages
    return tf.train.Example(features = tf.train.Features(feature = {
        'image/encoded':bytes_featurn(image_data),
        'image/format':bytes_featurn(image_format),
        'image/class/label':int64_feature(class_id)
    }))

def write_label_file(labels_to_class_names,dataset_dir,filename = LABELS_FILENAME):
    labels_filename = os.path.join(dataset_dir,filename)
    with tf.gfile.Open(labels_filename,'w') as f:
        for label in labels_to_class_names:
            class_name = labels_to_class_names[label]
            f.write('%d:%s\n'%(label,class_name))  



#把数据转为TFRecord格式  
def _convert_dataset(split_name,filenames,class_names_to_ids,dataset_dir):  
    # assert 断言 assert expression 相当于 if not expression raise AssertionError 
    assert split_name in ['train','test']   # 先判断一下split_name是不是训练集或者测试集
    
    # 计算每个数据块有多少个数据  
    # 这里是把数据块做了切分 当数据量非常大如imagenet时，切分数据块是必要的 切分放在多片tfrecord中
    # 本例无需切分 为距离说明 分_NUM_SHARDS=5片 放在多个tfrecord中
    num_per_shard = int(len(filenames) / _NUM_SHARDS)# 一片中的数据量 = 所有数据量/5取整
    with tf.Graph().as_default():
        with tf.Session() as sess:
            for shard_id in range(_NUM_SHARDS):
                #定义tfrecord文件的路径+名字  
                output_filename = _get_dataset_filename(dataset_dir,split_name,shard_id)
                
                # 这句是一个固定套路 把我们定义好的tfrecord文件的路径+名字 传入
                with tf.python_io.TFRecordWriter(output_filename) as tfrecore_writer:
                    #每一个数据块开始的位置  
                    start_ndx = shard_id * num_per_shard
                    #每一个数据块最后的位置  
                    end_ndx = min((shard_id+1) * num_per_shard,len(filenames))  
  
                    for i in range(start_ndx,end_ndx):
                        try:   # 防止有些图片可能损坏 打印出来跳过就行
                            sys.stdout.write('\r>>%s  Converting image %d/%d shard %d' % (split_name, i+1,len(filenames),shard_id))  
                            sys.stdout.flush()   # 这里打印一些信息 处理到了当前哪个图片 数据块是哪个数据块
                            # 读取图片  路径
                            image_data = tf.gfile.FastGFile(filenames[i],'rb').read()  
#                             img = Image.open(filenames[i])  
                            #img = img.resize((224, 224))  
#                             img_raw = img.tobytes()  
                            # 获取图片的类别名称  路径下最后一个文件夹的名字 就是分类名
                            class_name = os.path.basename(os.path.dirname(filenames[i]))
                            # 根据图片的分类可以找到类别名称对应的id  第几类
                            class_id = class_names_to_ids[class_name]  
                            #生成tfrecord文件       # 图片信息 格式 第几类
                            example = image_to_tfexample(image_data, b'jpg',class_id)  
                           # print(filenames[i]) 
                            
                            # 写入
                            tfrecore_writer.write(example.SerializeToString())  
                        except IOError as e:  
                            print("Could not read: ",filenames[i])  
                            print("Error: ",e)  
                            print("Skip it \n")  
  
    sys.stdout.write('\n')  
    sys.stdout.flush()  
    
    
    
if __name__=='__main__':  
    # 判断tfrecord文件是否存在  如果存在 就直接跳过了 不用管它了
    if _dataset_exists(DATASET_DIR):  
        print('tfrecord 文件已经存在')
    # 假如刚开始是没有这样的路径的
    else :  
        # 调用这个函数 获取图片文件夹中所有图片的 绝对路径列表 以及类名列表 class_names = []
        photo_filenames,class_names = _get_filenames_and_classes(DATASET_DIR)  
#         print(class_names)  
        #把分类转为字典格式 ，类似于{'house':0,'flower':1,'plane':4}  
        class_names_to_ids = dict(zip(class_names,range(len(class_names))))  
#         print(class_names_to_ids)  

        #把数据切为训练集和测试集  
        random.seed(_RANDOM_SEED)     # 给一个种子 看不懂
        random.shuffle(photo_filenames)    # 调用shuffle 可以把list打乱
        testing_filenames = photo_filenames[:_NUM_TEST]  # 分配给测试集
        training_filenames = photo_filenames[_NUM_TEST:]  # 分配给训练集

        #数据转换  （你是训练数据还是测试数据、训练集或是测试集、分类字典、图片路径）
        _convert_dataset('train',training_filenames,class_names_to_ids,DATASET_DIR)  
        _convert_dataset('test',testing_filenames,class_names_to_ids,DATASET_DIR)  

        #输出labels文件
        labels_to_class_names = dict(zip(range(len(class_names)),class_names))
        write_label_file(labels_to_class_names,DATASET_DIR)
        
        

>>train  Converting image 995/998 shard 4
>>test  Converting image 500/500 shard 4
